In [1]:
# Needed imports for opening sound tracks
import numpy as np
import pandas as pd
from IPython.display import Audio
from scipy.io import wavfile
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import tarfile
import librosa as lb
from librosa import feature, effects
import csv

In [2]:
#Downloading and extracting tar files
dataroot = "dataroot/"
!mkdir dataroot 
!wget -P dataroot http://opihi.cs.uvic.ca/sound/genres.tar.gz
with tarfile.open(dataroot+"genres.tar.gz","r:gz") as tar_ref:
    tar_ref.extractall(dataroot)
    tar_ref.close()

A subdirectory or file dataroot already exists.
--2021-10-18 21:24:23--  http://opihi.cs.uvic.ca/sound/genres.tar.gz
Resolving opihi.cs.uvic.ca (opihi.cs.uvic.ca)... 142.104.68.135
Connecting to opihi.cs.uvic.ca (opihi.cs.uvic.ca)|142.104.68.135|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1225571541 (1.1G) [application/x-gzip]
Saving to: 'dataroot/genres.tar.gz.2'

     0K .......... .......... .......... .......... ..........  0%  172K 1h55m
    50K .......... .......... .......... .......... ..........  0%  403K 82m36s
   100K .......... .......... .......... .......... ..........  0% 1002K 61m42s
   150K .......... .......... .......... .......... ..........  0%  683K 53m35s
   200K .......... .......... .......... .......... ..........  0% 1.10M 46m24s
   250K .......... .......... .......... .......... ..........  0% 1.31M 41m9s
   300K .......... .......... .......... .......... ..........  0% 1.34M 37m21s
   350K .......... .......... .......... ....

138450K .......... .......... .......... .......... .......... 11% 1.05M 28m54s
138500K .......... .......... .......... .......... .......... 11%  843K 28m53s
138550K .......... .......... .......... .......... .......... 11%  491K 28m53s
138600K .......... .......... .......... .......... .......... 11% 53.8M 28m53s
138650K .......... .......... .......... .......... .......... 11%  572K 28m53s
138700K .......... .......... .......... .......... .......... 11%  803K 28m53s
138750K .......... .......... .......... .......... .......... 11%  271K 28m53s
138800K .......... .......... .......... .......... .......... 11%  791K 28m53s
138850K .......... .......... .......... .......... .......... 11%  399K 28m53s
138900K .......... .......... .......... .......... .......... 11%  395K 28m53s
138950K .......... .......... .......... .......... .......... 11%  391K 28m54s
139000K .......... .......... .......... .......... .......... 11%  407K 28m54s
139050K .......... .......... ..........

199600K .......... .......... .......... .......... .......... 16%  418K 26m32s
199650K .......... .......... .......... .......... .......... 16%  555K 26m32s
199700K .......... .......... .......... .......... .......... 16%  555K 26m32s
199750K .......... .......... .......... .......... .......... 16%  402K 26m32s
199800K .......... .......... .......... .......... .......... 16%  700K 26m32s
199850K .......... .......... .......... .......... .......... 16%  563K 26m32s
199900K .......... .......... .......... .......... .......... 16%  568K 26m32s
199950K .......... .......... .......... .......... .......... 16%  608K 26m32s
200000K .......... .......... .......... .......... .......... 16%  466K 26m32s
200050K .......... .......... .......... .......... .......... 16%  392K 26m32s
200100K .......... .......... .......... .......... .......... 16%  405K 26m32s
200150K .......... .......... .......... .......... .......... 16%  644K 26m32s
200200K .......... .......... ..........

292350K .......... .......... .......... .......... .......... 24%  777K 23m37s
292400K .......... .......... .......... .......... .......... 24% 1.00M 23m37s
292450K .......... .......... .......... .......... .......... 24% 3.09M 23m37s
292500K .......... .......... .......... .......... .......... 24%  683K 23m37s
292550K .......... .......... .......... .......... .......... 24%  710K 23m37s
292600K .......... .......... .......... .......... .......... 24%  450K 23m37s
292650K .......... .......... .......... .......... .......... 24% 1.19M 23m37s
292700K .......... .......... .......... .......... .......... 24%  641K 23m36s
292750K .......... .......... .......... .......... .......... 24%  297K 23m37s
292800K .......... .......... .......... .......... .......... 24% 1.35M 23m36s
292850K .......... .......... .......... .......... .......... 24%  327K 23m37s
292900K .......... .......... .......... .......... .......... 24%  361K 23m37s
292950K .......... .......... ..........

402700K .......... .......... .......... .......... .......... 33%  792K 20m29s
402750K .......... .......... .......... .......... .......... 33% 1.11M 20m29s
402800K .......... .......... .......... .......... .......... 33%  803K 20m29s
402850K .......... .......... .......... .......... .......... 33%  773K 20m29s
402900K .......... .......... .......... .......... .......... 33%  857K 20m29s
402950K .......... .......... .......... .......... .......... 33%  819K 20m29s
403000K .......... .......... .......... .......... .......... 33% 2.17M 20m29s
403050K .......... .......... .......... .......... .......... 33% 1.10M 20m28s
403100K .......... .......... .......... .......... .......... 33%  814K 20m28s
403150K .......... .......... .......... .......... .......... 33%  801K 20m28s
403200K .......... .......... .......... .......... .......... 33% 1.29M 20m28s
403250K .......... .......... .......... .......... .......... 33% 1.54M 20m28s
403300K .......... .......... ..........

...

656750K .......... .......... .......... .......... .......... 54%  791K 13m16s
656800K .......... .......... .......... .......... .......... 54% 2.47M 13m16s
656850K .......... .......... .......... .......... .......... 54% 1.07M 13m16s
656900K .......... .......... .......... .......... .......... 54%  793K 13m16s
656950K .......... .......... .......... .......... .......... 54%  807K 13m16s
657000K .......... .......... .......... .......... .......... 54%  807K 13m16s
657050K .......... .......... .......... .......... .......... 54%  805K 13m15s
657100K .......... .......... .......... .......... .......... 54%  828K 13m15s
657150K .......... .......... .......... .......... .......... 54%  821K 13m15s
657200K .......... .......... .......... .......... .......... 54% 2.63M 13m15s
657250K .......... .......... .......... .......... .......... 54% 1.07M 13m15s
657300K .......... .......... .......... .......... .......... 54%  833K 13m15s
657350K .......... .......... ..........

780400K .......... .......... .......... .......... .......... 65%  821K 10m4s
780450K .......... .......... .......... .......... .......... 65%  896K 10m4s
780500K .......... .......... .......... .......... .......... 65% 7.84M 10m4s
780550K .......... .......... .......... .......... .......... 65%  811K 10m4s
780600K .......... .......... .......... .......... .......... 65%  835K 10m4s
780650K .......... .......... .......... .......... .......... 65%  880K 10m4s
780700K .......... .......... .......... .......... .......... 65% 7.77M 10m4s
780750K .......... .......... .......... .......... .......... 65%  837K 10m4s
780800K .......... .......... .......... .......... .......... 65%  886K 10m4s
780850K .......... .......... .......... .......... .......... 65% 2.27M 10m3s
780900K .......... .......... .......... .......... .......... 65% 1.08M 10m3s
780950K .......... .......... .......... .......... .......... 65%  863K 10m3s
781000K .......... .......... .......... .......... 

911500K .......... .......... .......... .......... .......... 76%  409K 6m40s
911550K .......... .......... .......... .......... .......... 76%  403K 6m40s
911600K .......... .......... .......... .......... .......... 76%  645K 6m40s
911650K .......... .......... .......... .......... .......... 76%  471K 6m40s
911700K .......... .......... .......... .......... .......... 76%  489K 6m40s
911750K .......... .......... .......... .......... .......... 76%  599K 6m40s
911800K .......... .......... .......... .......... .......... 76%  795K 6m40s
911850K .......... .......... .......... .......... .......... 76%  491K 6m39s
911900K .......... .......... .......... .......... .......... 76%  601K 6m39s
911950K .......... .......... .......... .......... .......... 76%  406K 6m39s
912000K .......... .......... .......... .......... .......... 76%  870K 6m39s
912050K .......... .......... .......... .......... .......... 76%  391K 6m39s
912100K .......... .......... .......... .......... 

1046200K .......... .......... .......... .......... .......... 87%  789K 3m29s
1046250K .......... .......... .......... .......... .......... 87%  792K 3m29s
1046300K .......... .......... .......... .......... .......... 87%  446K 3m29s
1046350K .......... .......... .......... .......... .......... 87%  412K 3m29s
1046400K .......... .......... .......... .......... .......... 87%  562K 3m29s
1046450K .......... .......... .......... .......... .......... 87%  512K 3m29s
1046500K .......... .......... .......... .......... .......... 87%  638K 3m29s
1046550K .......... .......... .......... .......... .......... 87%  462K 3m29s
1046600K .......... .......... .......... .......... .......... 87%  162K 3m29s
1046650K .......... .......... .......... .......... .......... 87%  190K 3m29s
1046700K .......... .......... .......... .......... .......... 87%  298K 3m29s
1046750K .......... .......... .......... .......... .......... 87%  403K 3m28s
1046800K .......... .......... .........

1172200K .......... .......... .......... .......... .......... 97% 1.16M 34s
1172250K .......... .......... .......... .......... .......... 97%  506K 34s
1172300K .......... .......... .......... .......... .......... 97%  803K 34s
1172350K .......... .......... .......... .......... .......... 97% 1.25M 34s
1172400K .......... .......... .......... .......... .......... 97% 1.02M 34s
1172450K .......... .......... .......... .......... .......... 97%  803K 34s
1172500K .......... .......... .......... .......... .......... 97%  831K 34s
1172550K .......... .......... .......... .......... .......... 97%  802K 34s
1172600K .......... .......... .......... .......... .......... 97%  822K 34s
1172650K .......... .......... .......... .......... .......... 97%  824K 34s
1172700K .......... .......... .......... .......... .......... 97%  838K 33s
1172750K .......... .......... .......... .......... .......... 97% 2.55M 33s
1172800K .......... .......... .......... .......... .......... 

In [3]:
#Creation of the csv file with the feature names and genre with numerical classification
music_data = []
music_target = []
genres = {'blues':0,'classical':1,'country':2,'disco':3,'hiphop':4,'jazz':5,'metal':6,'pop':7,'reggae':8,'rock':9}
header = ['tempo','harmonics','chroma_stft', 'rmse', 'centroid', 'bandwidth','contrast','flatness','rolloff', 'zero_crossing_rate']
for i in range(1, 21):
    header.append("mfcc" + str(i))
header.append("genre")



file = open('input_data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)


In [4]:
#Feature extraction using librosa and writing to the csv file for convienence

for genre, genre_num in genres.items():
    print(genre)
    for i in range(0,100):
        if i < 10:
            search = 'dataroot/genres/'+str(genre)+'/'+str(genre)+'.0000'+str(i)+'.wav'
        else:
            search = 'dataroot/genres/'+str(genre)+'/'+str(genre)+'.000'+str(i)+'.wav'
    
        y, sr = lb.load(search, mono=True, duration=30, sr=22050)
        
        row = []
        row.append(float(lb.beat.tempo(y=y, sr=sr)[0]))
        row.append(np.mean(effects.harmonic(y=y)))
        row.append(np.mean(feature.chroma_stft(y=y, sr=sr)))
        row.append(np.mean(feature.rms(y=y)))
        row.append(np.mean(feature.spectral_centroid(y=y, sr=sr)))
        row.append(np.mean(feature.spectral_bandwidth(y=y, sr=sr)))
        row.append(np.mean(feature.spectral_contrast(y=y, sr=sr)))
        row.append(np.mean(feature.spectral_flatness(y=y)))
        row.append(np.mean(feature.spectral_rolloff(y=y, sr=sr)))
        row.append(np.mean(feature.zero_crossing_rate(y)))
        
        mfcc = feature.mfcc(y=y, sr=sr)
        for e in mfcc:
            row.append(np.mean(e))
            
        row.append(genre_num)
        
        file = open('input_data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(row)
    print("Genre completed!")

blues
Genre completed!
classical
Genre completed!
country
Genre completed!
disco
Genre completed!
hiphop
Genre completed!
jazz
Genre completed!
metal
Genre completed!
pop
Genre completed!
reggae
Genre completed!
rock
Genre completed!


In [5]:
#Creation of a dataframe from the csv file
music_data = pd.read_csv('input_data.csv')
music_df = pd.DataFrame(music_data)
music_df

,tempo,harmonics,chroma_stft,rmse,centroid,bandwidth,contrast,flatness,rolloff,zero_crossing_rate,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,genre
0,123.046875,-4.903137e-05,0.349943,0.130225,1784.420446,2002.650192,20.530733,0.004499,3806.485316,0.083066,...,8.810669,-3.667368,5.751691,-5.162763,0.750948,-1.691938,-0.409953,-2.300209,1.219929,0
1,107.666016,1.418874e-04,0.340983,0.095918,1529.835316,2038.617579,20.676334,0.002294,3548.820207,0.056044,...,5.376803,-2.239120,4.216963,-6.012273,0.936110,-0.716537,0.293876,-0.287431,0.531573,0
2,161.499023,-1.297419e-06,0.363603,0.175573,1552.481958,1747.165985,22.197265,0.002628,3040.514948,0.076301,...,5.789265,-8.905224,-1.083720,-9.218360,2.455806,-7.726901,-1.815723,-3.433434,-2.226821,0
3,172.265625,4.781563e-07,0.404779,0.141191,1070.119953,1596.333948,21.428764,0.000953,2185.028454,0.033309,...,6.087676,-2.476421,-1.073890,-2.874778,0.780977,-3.316932,0.637982,-0.619690,-3.408233,0
4,135.999178,-1.798122e-05,0.308590,0.091563,1835.494603,1748.362448,21.467739,0.003239,3580.945013,0.101500,...,-2.806384,-6.934123,-7.558618,-9.173553,-4.512165,-5.453538,-0.924161,-4.409333,-11.703781,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,103.359375,3.591791e-06,0.351991,0.079469,2008.581132,2106.617024,22.988721,0.006553,4254.215942,0.089267,...,12.350818,-12.306271,5.780973,-10.279924,1.791489,-13.304210,2.473193,-6.717574,-1.189238,9
996,117.453835,1.402224e-06,0.398653,0.076452,2006.051164,2068.327905,22.673572,0.006453,4147.374921,0.097659,...,14.065345,-16.394691,4.152589,-4.350760,3.736455,-10.845638,1.875218,-7.459579,-2.802677,9
997,129.199219,-3.683992e-07,0.432103,0.081617,2077.190361,1926.989678,22.320874,0.007514,4030.767293,0.121824,...,10.481961,-17.951916,1.211114,-11.534864,1.844774,-12.847901,3.447425,-12.594178,-2.107002,9
998,112.347147,5.110090e-07,0.362349,0.083888,1398.672358,1818.148469,22.499462,0.001715,3014.740104,0.048731,...,7.683083,-10.071786,-0.255098,-5.276486,-2.816289,-4.416438,1.558265,-5.043121,-3.585596,9


In [6]:
#Separation of features and target from data frame and use of train test split and standard scaler
features = music_df.drop('genre',axis=1)
target = music_df['genre'].copy()

X_train, X_test, y_train, y_test = train_test_split(features,target, test_size=0.3, random_state=42)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
#Implementation of base RandomForestClassifier with training data then accuracy score on testing data
forest = RandomForestClassifier(random_state=42).fit(X_train,y_train)
y_pred = forest.predict(X_test)
accuracy_score(y_test,y_pred)

0.6333333333333333

In [11]:
#Implementation of KNeighborsClassifier with training data and accuracy check on testing data
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train,y_train)
knn.score(X_test,y_test)

0.5833333333333334

In [16]:
#Hyper tuning of RandomForestClassifier model to yield highest accuracy score as it yielded a higher accuracy than KNN
from sklearn.model_selection import GridSearchCV

max_depth = [x for x in range(10,110,10)]
max_depth.append(None)

params = {'n_estimators':[x for x in range(100,2100,100)],'max_depth':max_depth,'max_features':['auto','sqrt'],'bootstrap':[True,False]}

clf = GridSearchCV(estimator=forest, param_grid=params, n_jobs=-1)
clf.fit(X_train,y_train)
best_model = clf.best_estimator_

In [18]:
#Best parameters from hyper tuning
clf.best_params_

{'bootstrap': False,
 'max_depth': 20,
 'max_features': 'auto',
 'n_estimators': 200}

In [19]:
#Accuracy of testing set
best_model.score(X_test,y_test)

0.6633333333333333

In [24]:
#Accuracy of training set
best_model.score(X_train,y_train)

1.0

In [34]:
#Use of the model on personal jazz track, model classified it as Rock
y, sr = lb.load("Custom_Jazz_Track.wav", mono=True, duration=30, sr=22050)
        
row = []
row.append(float(lb.beat.tempo(y=y, sr=sr)[0]))
row.append(np.mean(effects.harmonic(y=y)))
row.append(np.mean(feature.chroma_stft(y=y, sr=sr)))
row.append(np.mean(feature.rms(y=y)))
row.append(np.mean(feature.spectral_centroid(y=y, sr=sr)))
row.append(np.mean(feature.spectral_bandwidth(y=y, sr=sr)))
row.append(np.mean(feature.spectral_contrast(y=y, sr=sr)))
row.append(np.mean(feature.spectral_flatness(y=y)))
row.append(np.mean(feature.spectral_rolloff(y=y, sr=sr)))
row.append(np.mean(feature.zero_crossing_rate(y)))

mfcc = feature.mfcc(y=y, sr=sr)
for e in mfcc:
    row.append(np.mean(e))
    
X = np.array(row).reshape(1, -1)    
best_model.predict(X)

array([9], dtype=int64)

In [ ]:
|